In [ ]:
from __future__ import absolute_import
from IPython.display import clear_output
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from os.path import exists, join

contentPath = os.getcwd()
cv2Path=join(contentPath, 'CV2Images')
testPath=join(contentPath, 'images')
checkpointPath = join(contentPath, 'CheckPoints')
print('ok')

In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0, patience=1, verbose=1,
    mode='auto', baseline=None, restore_best_weights=False,
    # print('\n',
)

checkpoints = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpointPath,
    monitor='loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)

In [ ]:
IMAGE_SIZE=(224, 224)
BATCH_SIZE = 2

import tensorflow
from tensorflow.keras.utils import image_dataset_from_directory
train_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    shuffle=True,
    batch_size=BATCH_SIZE)
print()

validation_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    shuffle=True,
    batch_size=BATCH_SIZE)
print()

test_ds = image_dataset_from_directory(
    testPath,
    color_mode='rgb',
    image_size=(224, 224),
    batch_size = BATCH_SIZE)
# clear_output()


In [ ]:
TRAIN_STEPS = 173 // BATCH_SIZE
print('TRAIN_STEPS:', TRAIN_STEPS)
VAL_STEPS =  43 // BATCH_SIZE
print('VAL_STEPS:', VAL_STEPS)

In [ ]:
bs='''
model = tf.keras.Sequential(name='Defcon4_Sequential_V3')
model.add(tf.keras.Input(shape=(224, 224, 3)))  # 224x224 RGB images
'''

In [ ]:
# Optionally, the first layer can receive an `input_shape` argument:

model = tf.keras.Sequential(name='Defcon4_Sequential_V4')
model.add(tf.keras.Input(shape=(224, 224, 3)))  # 224x224 RGB images

# Afterwards, we do automatic shape inference:
model.add(tf.keras.layers.Dense(4))

# This is identical to the following:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(16,)))
model.add(tf.keras.layers.Dense(8))

# Note that you can also omit the `input_shape` argument.
# In that case the model doesn't have any weights until the first call
# to a training/evaluation method (since it isn't yet built):
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(4))
# model.weights not created yet

# Whereas if you specify the input shape, the model gets built
# continuously as you are adding layers:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8, input_shape=(16,)))
model.add(tf.keras.layers.Dense(4))
len(model.weights)
# Returns "4"

# When using the delayed-build pattern (no input shape specified), you can
# choose to manually build your model by calling
# `build(batch_input_shape)`:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(4))
model.build((None, 16))
len(model.weights)
# Returns "4"

# Note that when using the delayed-build pattern (no input shape specified),
# the model gets built the first time you call `fit`, `eval`, or `predict`,
# or the first time you call the model on some input data.
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='sgd', loss='mse')
# This builds the model for the first time:
model.fit(x, y, batch_size=32, epochs=10)

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
model.built

In [ ]:
model.fit(
    x=train_ds,
    y=None,
    batch_size=BATCH_SIZE,
    epochs=5,
    verbose='auto',
    callbacks=[earlyStop, checkpoints],
    validation_split=0.0,
    validation_data=validation_ds,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=TRAIN_STEPS,
    validation_steps=VAL_STEPS,
    validation_batch_size=BATCH_SIZE,
    validation_freq=1,
    max_queue_size=2,
    workers=1,
    use_multiprocessing=True,
)

In [ ]:
modelName = model.name + '.h5'
modelSavePath = join(contentPath, modelName)
model.save(modelSavePath, overwrite=True)

In [ ]:
%ls